In [ ]:
import os
import json
import pandas as pd
import numpy as np
import tensorflow as tf
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
import matplotlib.pyplot as plt

# Load configuration from JSON file
config_path = '/content/drive/MyDrive/2024/University/FP/config_v6_86_acc.json'
with open(config_path, 'r') as f:
    config = json.load(f)

# Extract paths from config
base_dir = config['paths']['base_dir']
csv_path = config['paths']['csv_path']
img_dir = config['paths']['img_dir']
model_save_path = config['paths']['model_save_path']

# Check for GPU and set policy if compatible
if tf.config.list_physical_devices('GPU'):
    from tensorflow.keras import mixed_precision
    policy = mixed_precision.Policy('mixed_float16')
    mixed_precision.set_global_policy(policy)
else:
    print("Mixed precision not enabled: Compatible GPU not detected.")

# Load the CSV file
data = pd.read_csv(csv_path)

# Function to extract patch number from Patch_id
def get_patch_folder(patch_id):
    parts = patch_id.split('_')
    patch_num = parts[-1].split('.')[0]  # Extract patch number
    return f'Patch_{patch_num}'

# Debugging version of get_full_path
def get_full_path(row):
    try:
        patch_folder = get_patch_folder(row['Patch_id'])
        if row['label'] == 1:
            full_path = os.path.join(img_dir, 'mel_patches', patch_folder, row['Patch_id'])
        else:
            full_path = os.path.join(img_dir, 'bkl_patches', patch_folder, row['Patch_id'])

        if not os.path.exists(full_path):
            print(f"Invalid path: {full_path}")

        return full_path
    except Exception as e:
        print(f"Error processing row: {row}")
        print(f"Error message: {str(e)}")
        return None

# Apply the function to add the full path
data['path'] = data.apply(get_full_path, axis=1)

# Check for invalid paths and remove them
data['path_exists'] = data['path'].apply(os.path.exists)
invalid_paths = data[~data['path_exists']]
print(f"Number of invalid paths: {len(invalid_paths)}")
if not invalid_paths.empty:
    print(invalid_paths.head())
data = data[data['path_exists']]

# Convert label column to string
data['label'] = data['label'].astype(str)

# Split the data into training and validation sets
train_data, val_data = train_test_split(data, test_size=config['training']['validation_split'], stratify=data['label'], random_state=42)

# Use only a fraction of the data for training
train_data = train_data.sample(frac=config['training']['fraction'], random_state=42)
val_data = val_data.sample(frac=config['training']['fraction'], random_state=42)

# Image data generator for augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=config['data_augmentation']['rescale'],
    rotation_range=config['data_augmentation']['rotation_range'],
    width_shift_range=config['data_augmentation']['width_shift_range'],
    height_shift_range=config['data_augmentation']['height_shift_range'],
    shear_range=config['data_augmentation']['shear_range'],
    zoom_range=config['data_augmentation']['zoom_range'],
    horizontal_flip=config['data_augmentation']['horizontal_flip'],
    vertical_flip=config['data_augmentation']['vertical_flip'],
    brightness_range=config['data_augmentation']['brightness_range']
)

val_datagen = ImageDataGenerator(rescale=config['data_augmentation']['rescale'])

# Create training and validation generators
train_generator = train_datagen.flow_from_dataframe(
    train_data,
    x_col='path',
    y_col='label',
    target_size=(64, 64),
    batch_size=config['training']['batch_size'],
    class_mode='binary'
)

validation_generator = val_datagen.flow_from_dataframe(
    val_data,
    x_col='path',
    y_col='label',
    target_size=(64, 64),
    batch_size=config['training']['batch_size'],
    class_mode='binary'
)

# Define the pre-trained model with ResNet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

# Freeze the layers except the last 4 layers
for layer in base_model.layers[:-4]:
    layer.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(units=config['model_architecture']['dense_layers'][0]['units'], activation=config['model_architecture']['dense_layers'][0]['activation']),
    Dropout(rate=config['model_architecture']['dropout_rate']),
    Dense(1, activation='sigmoid')  # Sigmoid activation for binary classification
])

# Compile the model
initial_learning_rate = config['training']['initial_learning_rate']
optimizer = tf.keras.optimizers.Adam(learning_rate=initial_learning_rate)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Define callbacks
callbacks = [
    ModelCheckpoint(model_save_path, save_best_only=True, monitor='val_loss'),  # Monitor validation loss
    EarlyStopping(patience=config['training']['patience'], restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=config['training']['reduce_lr_factor'], patience=config['training']['reduce_lr_patience'], min_lr=config['training']['min_lr'])  # Reduce learning rate on plateau
]

# Train the model
history = model.fit(
    train_generator,
    epochs=config['training']['epochs'],
    validation_data=validation_generator,
    callbacks=callbacks,
    verbose=1
)

# Plot training & validation accuracy values
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()

# Define a function to pool predictions from the 16 patches to produce a final prediction for the full image
def predict_full_image(image_id, model, data):
    patches = data[data['image_id'] == image_id]['path'].values
    predictions = []

    for patch in patches:
        img = load_img(patch, target_size=(64, 64))
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0) / 255.0
        prediction = model.predict(img_array, verbose=0)
        predictions.append(prediction[0][0])

    return np.mean(predictions)

# Evaluate the model on the validation set
val_predictions = []
val_labels = []

for image_id in tqdm(val_data['image_id'].unique(), desc='Evaluating images'):
    label = val_data[val_data['image_id'] == image_id]['label'].values[0]
    val_labels.append(int(label))
    final_prediction = predict_full_image(image_id, model, val_data)
    val_predictions.append(final_prediction)

# Convert predictions to binary class (0 or 1)
val_predictions_binary = [1 if pred >= 0.5 else 0 for pred in val_predictions]

# Calculate accuracy
accuracy = np.mean(np.array(val_predictions_binary) == np.array(val_labels))
print('Validation accuracy:', accuracy)

# Example usage for a single image prediction
image_id = 'ISIC_0028965'
final_prediction = predict_full_image(image_id, model, data)
print(f'Final prediction for image {image_id}:', final_prediction)